Social coupling: mental health effect of COVID-19 measures
====

This notebook revolves around the analysis of data on the social and mental effect of the (Belgian) government COVID-19 measures.

## Tools

In [ ]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime

## PROMIS data metnal health
In this section, the investigation is on mental health data, in [PROMIS scale](http://www.healthmeasures.net/score-and-interpret/interpret-scores/promis/reference-populations), that was gathered during the lockdown by Melanie Beeckman and colleagues.

An overview of the questionnaire and its variables is given in `docs/social/survey_items_wave3.docx`.

### Reading

In [ ]:
# read data
rawdata = pd.read_csv('../data/raw/social/Gedrag_welzijn_data_WAVE3.csv', sep=';', encoding='utf-16')
# # create time stamps of data
for ind, val in enumerate(rawdata['datestamp']):
    if val[1]=='/': #correct for bad readings
        rawdata.loc[ind, 'datestamp'] = '0'+rawdata.loc[ind, 'datestamp']
# set index
rawdata.index = pd.to_datetime(rawdata['datestamp'], format='%m/%d/%Y %H:%M:%S')

# set values to numeric where possible
for c  in rawdata.columns:
    try:
        rawdata[c] = pd.to_numeric(rawdata[c], errors='coerce')
    except:
        pass

In [ ]:
# example plot of data
fig, ax = plt.subplots(figsize=(14,4))
ax.plot(rawdata['Slaap_tot_T'], '*')

In [ ]:
# check questionnaire variable names
for c in rawdata_wav1.columns:
    print(c)

### Analysis

In [ ]:
# select data for correlations
df4corr = rawdata_wav1
# compute correlations
dfcorr = df4corr.corr(method='spearman')
#select the highest correlations
dfcorr4plot = dfcorr#[np.abs(dfcorr)>0.6]

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))
ax = sns.heatmap(dfcorr4plot, ax=ax, cmap='BrBG', annot=False, vmin=-1, vmax=1, )
ax.tick_params(labelsize='large')